In [1]:
# !wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/ms.zip
# !unzip -o -qq en.zip
# !unzip -o -qq ms.zip

In [2]:
# !wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [3]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'OpenSubtitles/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'OpenSubtitles/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link id' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 2032182/2032182 [08:33<00:00, 3959.87it/s]


In [11]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            r = ' '.join([right_dict[k] for k in i[0]])
            l = ' '.join([left_dict[k] for k in i[1]])
            combined.append((l, r))
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

100%|██████████| 1769/1769 [10:37<00:00,  2.77it/s]


In [14]:
import json

with open('opensubtitle-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [15]:
!rm -rf OpenSubtitles
!rm -rf ms.zip en.zip